# Causal Fairness Analysis

In this tutorial, we will be exploring a causal fairness analysis, based ona. real-world dataset. Out theoretical framework would be Structural Theory of Causation and we will analyse based on existing work "Causal Fairness Analysis" by Plecko and Bareinboim (2022), and our own additional inferences. For test data, we will use ProPoblica COMPAS dataset. The eventual goal is to transform the idea for our own problem domain and dataset.

We will generate a computational pipeline, present it theoretically and show the data analysis in parallel.

## Questions /concerns

1. ***(RQ1)*** Bias Prevalence and Quantification
    1.1 Is there a bias - bias prevalence
    1.2 How to measure bias - bias quantification
    
2. ***(RQ2)*** Policy Adjustment for Bias Minimization 
    2.1 What policy recommendation for change required to minimize bias

## Our pipeline
1. Build a question: 
    1.1 Frame problem statement
    1.2 Frame variables
    1.3 Define structural causal model (SCM)
    1.3 Define structural fairness model SFM (X, Y, Z, W)
2. Answer causally: 
    2.1 Find answers to RQ1
    2.2 Find answers to RQ2

## Analyze Drago's Fairness Cookbook

In [1]:
fairness_cookbook <- function(data, X, W, Z, Y, x0, x1,
                              nboot = 100, model = "ranger", ...) {

  y <- as.numeric(data[[Y]]) - is.factor(data[[Y]]) # need to check (!)

  # bootstrap subsamples
  boots <- lapply(
    seq_len(nboot),
    function(i) {

      ind <- sample.int(nrow(data), replace = TRUE)
      is0 <- data[[X]][ind] == x0
      ind0 <- ind[is0]
      ind1 <- ind[!is0]

      list(all = ind, id0 = ind0, id1 = ind1)

    }
  )
    
  # mean-sd helper
    # function to get mean and sd
  msd <- function(x1, t1, x2, t2) {

    ms <- vapply(
      boots,
      function(ids) {
        mean(x1[ids[[t1]]], na.rm = TRUE) -  mean(x2[ids[[t2]]], na.rm = TRUE)
      }, numeric(1L)
    )

    c(mean(ms), sd(ms))

  }

  # get TV
  tv <- msd(y, "id1", y, "id0")

  ### non-mediated estimates
  {

    data0 <- data1 <- data
    if (!is.factor(data[[X]])) {
      data[[X]] <- factor(data[[X]], levels = c(x0, x1))
    }
    data0[[X]] <- factor(x0, levels = c(x0, x1))
    data1[[X]] <- factor(x1, levels = c(x0, x1))

    form <- as.formula(paste(Y, "~", paste(c(X, Z), collapse = "+")))

    mux1 <- model_mean(form, data, data1, Y = Y,
                       probability = length(unique(data[[Y]])) == 2L)

    mux0 <- model_mean(form, data, data0, Y = Y,
                       probability = length(unique(data[[Y]])) == 2L)

    idx <- data[[X]] == x1

    if (length(Z) > 0) {

      propx1 <- model_propensity(
        as.formula(paste0(X, " ~ ", paste0(Z, collapse = "+"))), data, Y = X,
        xlvl = x1)
    } else {

      propx1 <- rep(mean(idx), nrow(data))
    }


    yx1 <- (data[[Y]] - mux1) * idx / propx1 + mux1
    yx0 <- (data[[Y]] - mux0) * (!idx) / (1 - propx1) + mux0

    extrm_idx <- propx1 < 0.01 | propx1 > 0.99
    yx1[extrm_idx] <- yx0[extrm_idx] <- NA
    if (mean(extrm_idx) > 0.02) {
      message(100 * mean(extrm_idx),
              "% of extreme P(x | z) probabilities.\n",
              "TE, ETT, Exp-SE, Ctf-SE estimates likely biased.")
    }

  }
  # get SE
  ctfse <- msd(yx1, "id0", y, "id1") # Ctf-SE_{x_1, x_0}(y) = pyx1_x0 - py_x1
  expse_x1 <- msd(y, "id1", yx1, "all") # py_x1 - pyx1
  expse_x0 <- msd(y, "id0", yx0, "all") # py_x0 - pyx0
  # get TE/ETT
  ett <- msd(yx1, "id0", y, "id0") # pyx1_x0 - py_x0
  te <- msd(yx1, "all", yx0, "all") # pyx1 - pyx0

  ### mediated
  {

    est_med <- doubly_robust_med(data[[X]], data[, Z], data[, W], data[[Y]],
                                 model = model)
    yx0 <- est_med[[1]]
    yx1 <- est_med[[2]]
    yx1wx0 <- est_med[[4]]
  }
  # get DE
  nde <- msd(yx1wx0, "all", yx0, "all") # NDE_{x_0, x_1}(y)
  ctfde <- msd(yx1wx0, "id0", y, "id0") # Ctf-DE_{x_0, x_1}(y | x_0)
  # get IE
  nie <- msd(yx1wx0, "all", yx1, "all") # NIE_{x_1, x_0}(y)
  ctfie <- msd(yx1wx0, "id0", yx1, "id0") # Ctf-IE_{x_1, x_0}(y | x_0)

  # output
  structure(
    list(measures = list(
      TV = tv, CtfDE = ctfde, CtfSE = ctfse, ETT = ett, CtfIE = ctfie,
      TE = te, NDE = nde, NIE = nie, ExpSE_x1 = expse_x1,
      ExpSE_x0 = expse_x0
    ), x0 = x0, x1 = x1, model = model, X = X, W = W, Z = Z, Y = Y,
    cl = match.call()),
    class = "faircause"
  )

}

SyntaxError: positional argument follows keyword argument (39711157.py, line 2)